In [77]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from fractions import Fraction as frac
from decimal import Decimal
import regex as re

In [2]:
url = 'https://maplestory.nexon.com/Guide/OtherProbability/cube/red'


In [94]:

def process_df(df,first=True,firstweights=None,primeprob=None):
    split = df.iloc[:,0].str.split(':',expand=True)
    split.columns = ['Stat','Value']
    split.Stat.loc[split.Value.str.contains('%',na=False)] = split.Stat.loc[split.Value.str.contains('%',na=False)] + "%"
    split.Value = split.Value.str.extract('([0-9]+)').astype('int',errors='ignore')
    split.fillna({'Value':1},inplace=True)
    split.Stat = split.Stat.str.strip()
    probs = df.iloc[:,1].apply(lambda x: float(x.strip('%'))/100)
    if first:
        reduced = probs/min(probs)
        weights = reduced.apply(lambda x: frac(Decimal(x)).limit_denominator(100) * max(reduced.apply(lambda x: frac(Decimal(x)).limit_denominator(100).denominator))).astype('int')
    else:
        weights = pd.Series(index=probs.index)
        weights[-len(firstweights):] = firstweights
        weights[:-len(firstweights)] = probs[:-len(firstweights)] / primeprob * sum(firstweights)
        weights = weights.apply(lambda x: frac(Decimal(x)).limit_denominator(20) * max(weights.apply(lambda x: frac(Decimal(x)).limit_denominator(20).denominator)))
    split['Weight'] = weights
    return split



In [87]:
def get_cube_weights(url):
    html = requests.get(url).content
    df_list = pd.read_html(html, match='STR')
    soup = BeautifulSoup(html, 'html.parser')
    table_properties = soup.find_all('div',{'class': 'cube_option'})
    cube_df = []
    for i in range(0,len(df_list)):
        metadata = re.findall('<span>(.+)</span>',str(table_properties[i]))
        if metadata[0] == '레어':
            no_rare_lines = len(df_list[i].iloc[:,0:2].dropna())
            normal_lines = df_list[i].iloc[:-no_rare_lines,2:4].dropna()
            minidf = process_df(normal_lines)
            minidf['Grade'] = 'Normal'
            minidf['Equip'] = metadata[1]
            minidf['Level Requirement'] = int(re.findall('([0-9]+)',metadata[2])[0])
            minidf = minidf[['Equip','Level Requirement','Grade','Stat','Value','Weight']]
            cube_df.append(minidf)
        primelines = df_list[i].iloc[:,0:2].dropna()
        minidf = process_df(primelines)
        minidf['Grade'] = metadata[0]
        minidf['Equip'] = metadata[1]
        minidf['Level Requirement'] = int(re.findall('([0-9]+)',metadata[2])[0])
        minidf = minidf[['Equip','Level Requirement','Grade','Stat','Value','Weight']]
        cube_df.append(minidf)

    alldf = pd.concat(cube_df,ignore_index=True)
    return alldf

In [121]:
cubedict = {'Red':'https://maplestory.nexon.com/Guide/OtherProbability/cube/red',
           'Black': 'https://maplestory.nexon.com/Guide/OtherProbability/cube/black',
           'Additional': 'https://maplestory.nexon.com/Guide/OtherProbability/cube/addi'}
df = []
for cube,url in cubedict.items():
    cubeweights = get_cube_weights(url)
    cubeweights['Cube'] = cube
    df.append(cubeweights)

masterlist = pd.concat(df,ignore_index=True)
masterlist.to_csv('weights_kr.csv',index=False)

In [120]:
# koreancolumns = masterlist[['Equip','Grade','Stat']]
# koreanwords = []
# for i in koreancolumns.columns:
#     unique = koreancolumns[i].unique()
#     koreanwords += [x for x in unique]
# with open('koreanwords.csv','w+') as f:
#     for j in koreanwords:
#         f.write('"'+j+'"\n')

pd.read_excel('C:/users/amphi/documents/book1.xlsx')

,kr,en
0,무기,Weapon
1,엠블렘,Emblem
2,"보조무기 (포스실드, 소울링 제외)",Secondary (except Force Shield/Soul Ring/Shield)
3,"포스실드, 소울링",Force Shield/Soul Ring
4,방패,Shield
...,...,...
94,공격 시 15% 확률로 95의 MP 회복,15% chance of recovering 95 MP on attack
95,공격 시 3% 확률로 47의 HP 회복,3% chance of recovering 47 HP on attack
96,공격 시 3% 확률로 47의 MP 회복,3% chance of recovering 47 MP on attack
97,공격 시 15% 확률로 85의 HP 회복,15% chance of recovering 85 HP on attack
